In [1]:
from datasets import load_dataset, load_metric
import pandas as pd 
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action
from datasets import DatasetDict, Dataset
import nltk
nltk.download('punkt')
import string
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MT5ForConditionalGeneration, AutoTokenizer, AutoModel, BertTokenizer
import numpy as np
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\f.tomassen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Load downstream task evaluation data
from datasets import load_dataset, load_metric

eval_df = load_dataset("csv", data_files='E:/Data_exploration/GitHub/paraphraser_code/data/benchmarks/annotated_sentences.csv')

Found cached dataset csv (C:/Users/f.tomassen/.cache/huggingface/datasets/csv/default-e94bb2abe5d4234e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
#Load UL2 Model
import torch

#model_name = "ul2_finetuning_v1/checkpoint-17500"
model_dir = 'E:/Data_exploration/Floris_training/models/lion_models/v4_used_in_research/Ul2_large_100k_v2/checkpoint-10000'

model_checkpoint = "yhavinga/ul2-large-dutch"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 1024

#grab the column with sentences to apply it to...
eval_sentences = eval_df['train']['source']

#ul2 prefix
prefix = ""

eval_sentences = [prefix+sentence for sentence in eval_sentences]

e:\Data_exploration\fine-tune\310_py\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
def predict_correction(input_texts, batch_size=8, max_length=256):
    decoded_outputs = []

    for i in range(0, len(input_texts), batch_size):
        batch_texts = input_texts[i:i + batch_size]
        #print(len(batch_texts))
        
        # Encode the inputs using the tokenizer and pad them to the same length
        encoded_inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")

        # Get the input IDs and attention masks
        input_ids = encoded_inputs["input_ids"].to(device)  # Move to GPU if available
        attention_mask = encoded_inputs["attention_mask"].to(device)  # Move to GPU if available

        # Generate the output
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length)

        # Decode the output
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)

        decoded_outputs.extend(decoded_output)
        torch.cuda.empty_cache()

    return decoded_outputs


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU device
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [5]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_checkpoint)

try: 
    max_length = config.max_position_embeddings
except:
    max_length = config.n_positions

lengths = []
for sentence in eval_sentences: #_prefixed
    lengths.append(len(sentence))
longest_string = max(lengths)
if longest_string < max_length:
    max_length = longest_string
print("Maximum input sequence length:", max_length)

Maximum input sequence length: 359


In [6]:
#Generate candidates
#LOWER BATCHSIZE IF OUT OF GPU MEMORY
candidates = predict_correction(eval_sentences, max_length=max_length, batch_size = 8)
baselines = eval_df['train']['source'] #make_sentence_df(eval_df['train'], 'Preprocessed')
targets = eval_df['train']['target']#make_sentence_df(eval_df['train'], 'Target')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.41 GiB already allocated; 0 bytes free; 3.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
i = 2
print('candidate: ', candidates[i])
print('source', baselines[i])
print('target: ', targets[i])

candidate:  Kleding die er hing heb ik in de wasmand gedaan.
source Kleding die erhing heb ik in de wasmand gedaan.
target:  Kleding die er hing heb ik in de wasmand gedaan.
